# Fast RCNN

**Fast R-CNN** es una mejora directa de la R-CNN clásica.  

En este caso, las regiones de interés (*ROIs*) ya no se recortan ni se procesan individualmente para extraer características.  
En su lugar, la imagen completa pasa una sola vez por la red convolucional, generando un **feature map**.  

Después, se obtienen las propuestas de **Selective Search** y se proyectan sus coordenadas sobre ese mapa de características para extraer las regiones correspondientes.  

Dado que cada ROI tiene un tamaño distinto, se aplica **ROI Pooling** para normalizarlas a una forma fija, de modo que puedan pasar por las capas fully-connected del *head*.  (En implementaciones modernas usamos ROI Align)

La red se entrena optimizando simultáneamente:
- la **pérdida de clasificación** (qué objeto hay en cada ROI), y  
- la **pérdida de regresión** (ajuste fino de los *bounding boxes*).  

Durante el entrenamiento **no se aplica NMS**, pero **sí se usa en la inferencia** para eliminar detecciones redundantes.  

Esta arquitectura mejora notablemente los tiempos respecto a R-CNN, aunque **Selective Search sigue siendo el cuello de botella principal**.  


Diferencias:

| Etapa     | R-CNN clásica                  | Fast-RCNN                   |
| --------- | ------------------------------ | ------------------------------------------- |
| Proposals | Selective Search               | Selective Search                 |
| ROIs      | No hay como tal                | Subconjunto de proposals usado por ROIPooling / Align |
| NMS       | Final (sobre predicciones SVM) | Final (sobre predicciones del head)         |


Conceptos:


| Concepto      | Qué es                                                    | Cómo se obtiene                                        | Cuántos hay              | Cómo se usa                                                |
| ------------- | --------------------------------------------------------- | ------------------------------------------------------ | ------------------------ | ---------------------------------------------------------- |
| **Proposals** | Candidatas “donde podría haber algo”                      | De un algoritmo como **Selective Search** o un **RPN** | Miles (≈2000 por imagen) | Entrada bruta del detector                                 |
| **ROIs**      | Subconjunto de proposals elegidas para entrenar o inferir | Se **muestrean** (pos/neg) de las proposals            | Pocas (≈128 por imagen)  | Alimentan **ROIAlign + head** para clasificación/regresión |


Vamos a ver su implementación

In [1]:
import os, random, math, time, pathlib, shutil, json
import numpy as np
import torch
import torchvision as tv

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("CUDA disponible:", torch.cuda.is_available())
print("Device:", device)

VOC_ROOT = os.environ.get("VOC_ROOT", "./data/02")
print("VOC_ROOT:", VOC_ROOT)


CUDA disponible: True
Device: cuda
VOC_ROOT: ./data/02


#### Nos saltamos la parte de descargar el dataset, lo vimos en el cuaderno anterior.

Pasamos directamente a su carga.

In [2]:
from torchvision.datasets import VOCDetection

train_root = "./data/02/VOCtrainval_06-Nov-2007"
test_root  = "./data/02/VOCtest_06-Nov-2007"

train_ds = VOCDetection(train_root, year="2007", image_set="trainval", download=False)
test_ds  = VOCDetection(test_root,  year="2007", image_set="test",     download=False)

print("Train:", len(train_ds))
print("Test:", len(test_ds))


Train: 5011
Test: 4952


In [ ]:
# === Extracción automática de clases del dataset VOC ===

def extract_voc_classes(dataset):
    classes = set()
    for i in range(min(500, len(dataset))):  # escanea solo 500 imágenes para acelerar
        ann = dataset[i][1]['annotation']
        objs = ann.get('object', [])
        if isinstance(objs, dict):  # si solo hay un objeto
            objs = [objs]
        for obj in objs:
            name = obj['name']
            classes.add(name)
    return sorted(list(classes))

# Obtener clases del dataset de entrenamiento
voc_classes = extract_voc_classes(train_ds)

#Añadimos background porque es la clase 0, que no está en voc_classes, lo necesitamos para el loss. 
#Tendremos 21 clases: 20 de VOC + 1 de background.
CLASSES = ['background'] + voc_classes

NUM_CLASSES = len(CLASSES)

print("Clases detectadas:", voc_classes)
print("Total:", NUM_CLASSES)


Clases detectadas: ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
Total: 21


Tenemos unas 2000 proposals por imagen
Estas salen de Selective Search.
La mayoría no contiene ningún objeto útil; algunas sí cubren bien GT.
El objetivo es quedarse con 128 ROIs representativas para esa imagen.

Para ello calculamos el IoU entre cada proposal y todas las cajas GT.

Luego clasificamos cada proposal en Pos y Neg en base a los thresholds del IoU y poderamos las imagenes que nos queremos quedar.


In [ ]:


# IoU thresholds
POS_IOU_THRESH = 0.5
NEG_IOU_THRESH = 0.3

# Sampler config
# Tomamos 128 ROIs por imagen.
ROIS_PER_IMG = 128
# 25% de esas ROIs son positivas (foreground) y el resto 75% negativas (background).
# Usamos esta distribución porque es lo que da mejores resultados en la literatura.
FG_FRACTION = 0.25  # ~32 pos + 96 neg 

# BBox delta normalization
# Normalizamos las coordenadas de los bounding boxes para que sean más estables.
#Son valores que usamos fijos para todo el entrenamiento y que se han obtenido empíricamente.
#Son un estándar en la literatura.
BBOX_MEANS = torch.tensor([0.0, 0.0, 0.0, 0.0], dtype=torch.float32)
BBOX_STDS  = torch.tensor([0.1, 0.1, 0.2, 0.2], dtype=torch.float32)

# Proposals
# Tomamos hasta 2000 propuestas por imagen.
MAX_PROPOSALS_PER_IMG = 2000
# Las propuestas mínimas tienen 16px de lado.
MIN_SIZE = 16

print(f"Num clases: {NUM_CLASSES}")
print(f"Sampler: {ROIS_PER_IMG} (FG {int(ROIS_PER_IMG*FG_FRACTION)}, BG {ROIS_PER_IMG-int(ROIS_PER_IMG*FG_FRACTION)})")
print(f"PosIoU≥{POS_IOU_THRESH}, NegIoU≤{NEG_IOU_THRESH}")

Num clases: 21
Sampler: 128 (FG 32, BG 96)
PosIoU≥0.5, NegIoU≤0.3
